## 1 Imports 

In [50]:
# imports
import numpy as np
import pandas as pd

# imports for nlp
import re
import os
import nltk
import string
import time
import spacy.cli

import unicodedata
import numpy as np
import pandas as pd
import xgboost
from sklearn import metrics
from nltk.corpus import stopwords
from html.parser import HTMLParser
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
nltk.download('stopwords')

# imports for models
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier

#imports for performance evaluation
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, classification_report, confusion_matrix
# Import library for train test split

from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [51]:
# Loading the data
train = pd.read_csv('train_set.csv')
test = pd.read_csv('test_set.csv')

## 2 EDA

In [52]:
# train data summary
train.head(20)

,lang_id,text
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...
1,xho,i-dha iya kuba nobulumko bokubeka umsebenzi na...
2,eng,the province of kwazulu-natal department of tr...
3,nso,o netefatša gore o ba file dilo ka moka tše le...
4,ven,khomishini ya ndinganyiso ya mbeu yo ewa maana...
5,nso,dinyakišišo tše tša go dirwa gabedi ka ngwaga ...
6,tsn,kgetse nngwe le nngwe e e sa faposiwang mo tsh...
7,ven,mbadelo dze dza laelwa dzi do kwama mahatulele...
8,nso,maloko a dikhuduthamaga a ikarabela mongwe le ...
9,tsn,fa le dirisiwa lebone le tshwanetse go bontsha...


In [53]:
# test data summary
test.head(20)

,index,text
0,1,"Mmasepala, fa maemo a a kgethegileng a letlele..."
1,2,Uzakwaziswa ngokufaneleko nakungafuneka eminye...
2,3,Tshivhumbeo tshi fana na ngano dza vhathu.
3,4,Kube inja nelikati betingevakala kutsi titsini...
4,5,Winste op buitelandse valuta.
5,6,"Ke feela dilense tše hlakilego, tša pono e tee..."
6,7,<fn>(762010101403 AM) 1495 Final Gems Birthing...
7,8,Ntjhafatso ya konteraka ya mosebetsi: Etsa bon...
8,9,u-GEMS uhlinzeka ngezinzuzo zemithi yezifo ezi...
9,10,"So, on occasion, are statistics misused."


In [54]:
text = train['text']
text

0        umgaqo-siseko wenza amalungiselelo kumaziko ax...
1        i-dha iya kuba nobulumko bokubeka umsebenzi na...
2        the province of kwazulu-natal department of tr...
3        o netefatša gore o ba file dilo ka moka tše le...
4        khomishini ya ndinganyiso ya mbeu yo ewa maana...
                               ...                        
32995    popo ya dipolateforomo tse ke go tlisa boetele...
32996    modise mosadi na o ntse o sa utlwe hore thaban...
32997    closing date for the submission of completed t...
32998    nawuphina umntu ofunyenwe enetyala phantsi kwa...
32999    mafapha a mang le ona a lokela ho etsa ditlale...
Name: text, Length: 33000, dtype: object

## Data Processing 


In [55]:
train1 = train.copy()

In [56]:
train1 = train.drop_duplicates(keep='first')
train1.shape

(29948, 2)

In [57]:
def message (text):
        
    #convert to lowercase
    text = text.lower()
     
    #remove numbers 
    text = re.sub(r'\d+','', text)
    
    #remove punctuation
    text = re.sub(r"[,.;':@#?!\&/$]+\ *", ' ', text)
    
    #remove extra whitespaces 
    text = re.sub(r'\s\s+', ' ', text)
    
    #remove space in front of tweet
    text = text.lstrip(' ')
    
    return text

In [58]:
train1['text_']= train1['text'].apply(message)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


## 3 Feature Engineering

splitting the data

In [59]:
X = train1['text_']
y = train1['lang_id']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.1, shuffle=True, stratify=y, random_state = 42)

Pipeline

In [60]:
#Logistic Regression
lr = Pipeline([('tfidf', TfidfVectorizer()),
               ('clf', LogisticRegression())])

#Linear SVC
linsvc = Pipeline([('tfidf', TfidfVectorizer()),
                   ('clf', LinearSVC())])


#Random Forest Classifier
rf = Pipeline([('tfidf', TfidfVectorizer()),
               ('clf', RandomForestClassifier())])


## 4 Modeling

Train the models

In [61]:
# Logistic regression
lr.fit(X_train, y_train) 
y_pred_lr = lr.predict(X_test) 

# Linear SVC
linsvc.fit(X_train, y_train)
y_pred_linsvc = linsvc.predict(X_test)

# Random Forest Classifier
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


logistic regression

In [62]:
start = time.time()
lg_f1 = round(f1_score(y_test, y_pred_lr, average='weighted'),2)
print('Accuracy %s' % accuracy_score(y_pred_lr, y_test))
print("Model Runtime: %0.2f seconds"%((time.time() - start)))
report = classification_report(y_test, y_pred_lr, output_dict=True)
results = pd.DataFrame(report).transpose()
results

Accuracy 0.9956594323873121
Model Runtime: 0.02 seconds


,precision,recall,f1-score,support
afr,1.000000,0.996212,0.998102,264.000000
eng,0.996678,1.000000,0.998336,300.000000
nbl,0.995708,0.983051,0.989339,236.000000
nso,1.000000,1.000000,1.000000,287.000000
sot,1.000000,1.000000,1.000000,283.000000
ssw,1.000000,0.995885,0.997938,243.000000
tsn,1.000000,1.000000,1.000000,287.000000
tso,1.000000,1.000000,1.000000,276.000000
ven,1.000000,1.000000,1.000000,261.000000
xho,0.981481,0.996241,0.988806,266.000000


linear svc

In [63]:
linsvc_f1 = round(f1_score(y_test, y_pred_linsvc, average='weighted'),2)
print('Accuracy %s' % accuracy_score(y_pred_linsvc, y_test))
print("Model Runtime: %0.2f seconds"%((time.time() - start)))
report = classification_report(y_test, y_pred_linsvc, output_dict=True)
results = pd.DataFrame(report).transpose()
results

Accuracy 0.998330550918197
Model Runtime: 0.16 seconds


,precision,recall,f1-score,support
afr,1.000000,1.000000,1.000000,264.000000
eng,0.996678,1.000000,0.998336,300.000000
nbl,1.000000,0.995763,0.997877,236.000000
nso,1.000000,1.000000,1.000000,287.000000
sot,1.000000,1.000000,1.000000,283.000000
ssw,1.000000,0.991770,0.995868,243.000000
tsn,1.000000,1.000000,1.000000,287.000000
tso,1.000000,1.000000,1.000000,276.000000
ven,1.000000,1.000000,1.000000,261.000000
xho,0.996255,1.000000,0.998124,266.000000


Random Forest

In [64]:
rf_f1 = round(f1_score(y_test, y_pred_rf, average='weighted'),2)
print('Accuracy %s' % accuracy_score(y_pred_rf, y_test))
print("Model Runtime: %0.2f seconds"%((time.time() - start)))
report = classification_report(y_test, y_pred_rf, output_dict=True)
pd.DataFrame(report).transpose()

Accuracy 0.9823038397328882
Model Runtime: 0.30 seconds


,precision,recall,f1-score,support
afr,0.996226,1.000000,0.998110,264.000000
eng,0.990099,1.000000,0.995025,300.000000
nbl,0.986486,0.927966,0.956332,236.000000
nso,1.000000,1.000000,1.000000,287.000000
sot,1.000000,1.000000,1.000000,283.000000
ssw,0.995575,0.925926,0.959488,243.000000
tsn,1.000000,1.000000,1.000000,287.000000
tso,1.000000,1.000000,1.000000,276.000000
ven,1.000000,1.000000,1.000000,261.000000
xho,0.977186,0.966165,0.971645,266.000000


## Hyperparameter Tuning

Multinomial Naive Bayes

In [65]:
#Multinomial Naive Bayes
param_grid={'alpha':[1,0.1,0.01,0.001,0.0001,0.00001]}
multi = Pipeline([('tfidf', TfidfVectorizer(ngram_range=(1,3),min_df=2, stop_words='english')),
                  ('clf', GridSearchCV(MultinomialNB(),
                                       param_grid=param_grid,
                                       cv =5,
                                       n_jobs=-1) )])

multi.fit(X_train, y_train)
y_pred_multi=multi.predict(X_test)

print(classification_report(y_test, y_pred_multi))
multin=classification_report(y_test, y_pred_multi, output_dict=True)


              precision    recall  f1-score   support

         afr       1.00      1.00      1.00       264
         eng       1.00      1.00      1.00       300
         nbl       1.00      1.00      1.00       236
         nso       1.00      1.00      1.00       287
         sot       1.00      1.00      1.00       283
         ssw       1.00      1.00      1.00       243
         tsn       1.00      1.00      1.00       287
         tso       1.00      1.00      1.00       276
         ven       1.00      1.00      1.00       261
         xho       1.00      1.00      1.00       266
         zul       1.00      1.00      1.00       292

    accuracy                           1.00      2995
   macro avg       1.00      1.00      1.00      2995
weighted avg       1.00      1.00      1.00      2995



## 5 Submission

In [66]:
df1 = test.copy()
df1

,index,text
0,1,"Mmasepala, fa maemo a a kgethegileng a letlele..."
1,2,Uzakwaziswa ngokufaneleko nakungafuneka eminye...
2,3,Tshivhumbeo tshi fana na ngano dza vhathu.
3,4,Kube inja nelikati betingevakala kutsi titsini...
4,5,Winste op buitelandse valuta.
...,...,...
5677,5678,You mark your ballot in private.
5678,5679,Ge o ka kgetha ka bowena go se šomiše Mofani k...
5679,5680,"E Ka kopo etsa kgetho ya hao ka hloko, hobane ..."
5680,5681,"TB ke bokudi ba PMB, mme Morero o tla lefella ..."


In [67]:
test_X = df1.text

In [68]:
y_pred =multi.predict(test_X)

In [69]:
df1['lang_id'] = y_pred

In [70]:
df1.head()

,index,text,lang_id
0,1,"Mmasepala, fa maemo a a kgethegileng a letlele...",tsn
1,2,Uzakwaziswa ngokufaneleko nakungafuneka eminye...,nbl
2,3,Tshivhumbeo tshi fana na ngano dza vhathu.,ven
3,4,Kube inja nelikati betingevakala kutsi titsini...,ssw
4,5,Winste op buitelandse valuta.,afr


In [72]:
df1[['index','lang_id']].to_csv('submissionf5.csv', index=False)